In [ ]:
!pip install zarr, fsspec, gcsfs
!pip install xarray[complete]
!pip install gcsfs
!pip install zarr
!pip install crc32c

In [5]:
import gcsfs
import xarray as xr
import zarr

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, fixed
from matplotlib.colors import TwoSlopeNorm

from datetime import datetime
import os
import json




---



Loading the dataset, with these parameters :   
    - Variable : temperature at 2m  
    - Geographical region : France  
    - Resolution : 1440x720  
    - Time step : 6h  

In [ ]:
# Dataset
dataset_path = "gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr"

try:
    fs = gcsfs.GCSFileSystem(token='anon')
    mapper = fs.get_mapper(dataset_path)
    ds = xr.open_zarr(mapper, chunks={'time': 24}, consolidated=True)

    # Excarct the variable temperature at 2m
    temperature_data = ds['2m_temperature']

    # EXTRACTION FRANCE
    lat_min, lat_max = 42.0, 51.5
    lon_min_converted = 355.0  # -5 + 360
    lon_max_converted = 9.0

    lat_values = temperature_data.latitude.values
    lon_values = temperature_data.longitude.values

    lat_mask = (lat_values >= lat_min) & (lat_values <= lat_max)
    lat_indices = np.where(lat_mask)[0]

    lon_mask1 = lon_values >= lon_min_converted  # 355° à 360°
    lon_mask2 = lon_values <= lon_max_converted  # 0° à 9°
    lon_indices1 = np.where(lon_mask1)[0]
    lon_indices2 = np.where(lon_mask2)[0]

    france_part1 = temperature_data.isel(latitude=lat_indices, longitude=lon_indices1)
    france_part2 = temperature_data.isel(latitude=lat_indices, longitude=lon_indices2)
    france_data = xr.concat([france_part1, france_part2], dim='longitude')

    # Size of data
    nbytes = france_data.nbytes
    size_str = f"{nbytes / 1e9:.2f} GB" if nbytes > 1e9 else f"{nbytes / 1e6:.2f} MB"
    print(f"\n Taille mémoire: {size_str}")

    # Coordinates
    lon_display = np.where(france_data.longitude.values > 180,
                          france_data.longitude.values - 360,
                          france_data.longitude.values)

    # Resolution
    lat_spacing = np.mean(np.diff(france_data.latitude.values))
    lon_spacing = np.mean(np.diff(france_data.longitude.values))

    # Hours disponibles
    hours = pd.to_datetime(france_data.time.values).hour
    unique_hours = np.unique(hours)


except Exception as e:
    print(f"\n Erreur: {e}")



In [7]:
france_data

<xarray.DataArray '2m_temperature' (time: 92044, latitude: 39, longitude: 57)> Size: 818MB
dask.array<concatenate, shape=(92044, 39, 57), dtype=float32, chunksize=(24, 39, 37), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 736kB 1959-01-01 ... 2021-12-31T18:00:00
  * latitude   (latitude) float32 156B 51.5 51.25 51.0 50.75 ... 42.5 42.25 42.0
  * longitude  (longitude) float32 228B 355.0 355.2 355.5 355.8 ... 8.5 8.75 9.0
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

Data split :  
    - Training data : from 2011 to 2016  
    - Validation data : 2018  
    - Test data : 2020 and 2021  

In [ ]:
test_data = france_data.sel(time=slice("2020-01-01", "2021-12-31"))
validation_data = france_data.sel(time=slice("2018-01-01", "2018-12-31"))
training_data = france_data.sel(time=slice("2011-01-01", "2016-12-31"))

# Size of training data
nbytes = training_data.nbytes
size_str = f"{nbytes / 1e9:.2f} GB" if nbytes > 1e9 else f"{nbytes / 1e6:.2f} MB"
print(f"\n Taille mémoire training_data : {size_str}")

Input and output split :  
    • Input <- 6 consequentive values of tempereture [t-36h ; t-30h ; t-24h ; t-18h ; t-12h ; t-6h]  
    • Output <- the 7th value of tempereture at the time t  

In [ ]:
INPUT_TIMESTEPS = 6   # 6 timesteps in input
OUTPUT_TIMESTEPS = 1  # 1 timestep in output
FORECAST_HORIZON = 1  # We predict the timestep immediately after

data_array_train = training_data.values
data_array_val = validation_data.values
data_array_test = test_data.values

# Input and output sequences :
X_sequences_train = []
Y_sequences_train = []

num_sequences_train = len(data_array_train) - INPUT_TIMESTEPS - FORECAST_HORIZON + 1
num_sequences_val = len(data_array_val) - INPUT_TIMESTEPS - FORECAST_HORIZON + 1
num_sequences_test = len(data_array_test) - INPUT_TIMESTEPS - FORECAST_HORIZON + 1

for i in range(num_sequences_train):
    x_seq = data_array_train[i : i + INPUT_TIMESTEPS]
    y_seq = data_array_train[i + INPUT_TIMESTEPS + FORECAST_HORIZON - 1]

    X_sequences_train.append(x_seq)
    Y_sequences_train.append(y_seq)

X_train = np.array(X_sequences_train)  
Y_train = np.array(Y_sequences_train)  

X = np.expand_dims(X_train, axis=1)
Y = np.expand_dims(Y_train, axis=1)

# Same for validation data
X_sequences_val = []
Y_sequences_val = []

num_sequences_val = len(data_array_val) - INPUT_TIMESTEPS - FORECAST_HORIZON + 1

for i in range(num_sequences_val):
    x_seq = data_array_val[i : i + INPUT_TIMESTEPS] 
    y_seq = data_array_val[i + INPUT_TIMESTEPS + FORECAST_HORIZON - 1] 

    X_sequences_val.append(x_seq)
    Y_sequences_val.append(y_seq)

X_val = np.array(X_sequences_val)  
Y_val = np.array(Y_sequences_val)  

X_val = np.expand_dims(X_val, axis=1)
Y_val = np.expand_dims(Y_val, axis=1)

# Same for test data
X_sequences_test = []
Y_sequences_test = []

num_sequences_test = len(data_array_test) - INPUT_TIMESTEPS - FORECAST_HORIZON + 1

for i in range(num_sequences_test):
    x_seq = data_array_test[i : i + INPUT_TIMESTEPS]
    y_seq = data_array_test[i + INPUT_TIMESTEPS + FORECAST_HORIZON - 1]
    X_sequences_test.append(x_seq)
    Y_sequences_test.append(y_seq)

X_test = np.array(X_sequences_test)
Y_test = np.array(Y_sequences_test)

X_test = np.expand_dims(X_test, axis=1)
Y_test = np.expand_dims(Y_test, axis=1)

X_train = X
Y_train = Y

# NORMALISATION
mean_temp = X_train.mean()
std_temp = X_train.std()

X_train_norm = (X_train - mean_temp) / std_temp
X_val_norm = (X_val - mean_temp) / std_temp
X_test_norm = (X_test - mean_temp) / std_temp

Y_train_norm = (Y_train - mean_temp) / std_temp
Y_val_norm = (Y_val - mean_temp) / std_temp
Y_test_norm = (Y_test - mean_temp) / std_temp

In [ ]:
#Saving the normalized data
output_dir = "./data/processed"
os.makedirs(output_dir, exist_ok=True)

np.save(f"{output_dir}/X_train.npy", X_train_norm)
np.save(f"{output_dir}/Y_train.npy", Y_train_norm)
np.save(f"{output_dir}/X_val.npy", X_val_norm)
np.save(f"{output_dir}/Y_val.npy", Y_val_norm)
np.save(f"{output_dir}/X_test.npy", X_test_norm)
np.save(f"{output_dir}/Y_test.npy", Y_test_norm)

# Normalization parameters
np.save(f"{output_dir}/norm_params.npy", np.array([mean_temp, std_temp]))

# Saving the metadata
metadata = {
    'input_timesteps': INPUT_TIMESTEPS,
    'output_timesteps': OUTPUT_TIMESTEPS,
    'forecast_horizon': FORECAST_HORIZON,
    'mean_temp': float(mean_temp),
    'std_temp': float(std_temp),
    'spatial_shape': (39, 57),
    'n_train': len(X_train),
    'n_val': len(X_val),
    'n_test': len(X_test)
}

with open(f"{output_dir}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

Visualization

In [ ]:
longitudes = training_data['longitude'].values
latitudes = training_data['latitude'].values
times = training_data['time'].values

data_np = data_array_train
# Convertir les longitudes > 180 pour l'affichage
lon_display = np.where(longitudes > 180, longitudes - 360, longitudes)


In [ ]:
def plot_temperature(t):
    
    timestamp = pd.Timestamp(times[t])
    date_str = timestamp.strftime('%Y-%m-%d %H:%M')

    temp_kelvin = data_np[t, :, :] 
    temp_celsius = temp_kelvin - 273.15 

    fig, ax = plt.subplots(figsize=(14, 8))

    contour = ax.contourf(
        lon_display,
        latitudes,
        temp_celsius,
        levels=20,
        cmap='RdYlBu_r',
        extend='both'
    )

    contour_lines = ax.contour(
        lon_display,
        latitudes,
        temp_celsius,
        levels=10,
        colors='black',
        linewidths=0.5,
        alpha=0.3
    )
    ax.clabel(contour_lines, inline=True, fontsize=8, fmt='%.1f°C')

    cbar = plt.colorbar(contour, ax=ax, orientation='vertical', pad=0.02)
    cbar.set_label('Température (°C)', fontsize=12)

    ax.set_title(f'Température France - {date_str}\nTimestep {t}/{len(times)-1}',
                 fontsize=14, fontweight='bold')
    ax.set_xlabel('Longitude (°E)', fontsize=12)
    ax.set_ylabel('Latitude (°N)', fontsize=12)

    ax.grid(True, alpha=0.3, linestyle='--')

    ax.set_xlim(lon_display.min(), lon_display.max())
    ax.set_ylim(latitudes.min(), latitudes.max())

    temp_min = temp_celsius.min()
    temp_max = temp_celsius.max()
    temp_mean = temp_celsius.mean()

    stats_text = f'Min: {temp_min:.1f}°C | Max: {temp_max:.1f}°C | Moyenne: {temp_mean:.1f}°C'
    ax.text(0.5, 0.02, stats_text,
            transform=ax.transAxes,
            ha='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=10)

    plt.tight_layout()
    plt.show()

def plot_sequence_6to1(start_idx):
    
    if start_idx > len(times) - 7:
        print(f" Index trop grand! Max: {len(times) - 7}")
        return

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()

    all_temps = []
    for i in range(7):
        temp_celsius = data_np[start_idx + i] - 273.15
        all_temps.append(temp_celsius)

    vmin = min([t.min() for t in all_temps])
    vmax = max([t.max() for t in all_temps])

    # Plot the 6 inputs
    for i in range(6):
        ax = axes[i]
        temp_celsius = data_np[start_idx + i] - 273.15
        timestamp = pd.Timestamp(times[start_idx + i])
        date_str = timestamp.strftime('%Y-%m-%d %H:%M')

        contour = ax.contourf(lon_display, latitudes, temp_celsius,
                              levels=20, cmap='RdYlBu_r',
                              vmin=vmin, vmax=vmax, extend='both')

        ax.set_title(f'INPUT {i+1}\n{date_str}', fontsize=10, fontweight='bold', color='blue')
        ax.set_xlabel('Lon', fontsize=8)
        ax.set_ylabel('Lat', fontsize=8)
        ax.grid(True, alpha=0.2)

    # Plot the output (7th timestep)
    ax = axes[6]
    temp_celsius = data_np[start_idx + 6] - 273.15
    timestamp = pd.Timestamp(times[start_idx + 6])
    date_str = timestamp.strftime('%Y-%m-%d %H:%M')

    contour = ax.contourf(lon_display, latitudes, temp_celsius,
                          levels=20, cmap='RdYlBu_r',
                          vmin=vmin, vmax=vmax, extend='both')

    ax.set_title(f'OUTPUT (Target)\n{date_str}', fontsize=10, fontweight='bold', color='red')
    ax.set_xlabel('Lon', fontsize=8)
    ax.set_ylabel('Lat', fontsize=8)
    ax.grid(True, alpha=0.2)

    fig.delaxes(axes[7])

    cbar = fig.colorbar(contour, ax=axes[:7], orientation='vertical',
                       pad=0.02, aspect=30)
    cbar.set_label('Température (°C)', fontsize=12)

    fig.suptitle(f'Séquence d\'entraînement 6→1 (Index: {start_idx})\n' +
                 f'6 timesteps input (36h) → 1 timestep output (+6h)',
                 fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

def plot_input_vs_output(start_idx):
    if start_idx > len(times) - 7:
        print(f" Index trop grand! Max: {len(times) - 7}")
        return

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    temp_input = data_np[start_idx + 5] - 273.15
    timestamp_input = pd.Timestamp(times[start_idx + 5])

    temp_output = data_np[start_idx + 6] - 273.15
    timestamp_output = pd.Timestamp(times[start_idx + 6])

    vmin = min(temp_input.min(), temp_output.min())
    vmax = max(temp_input.max(), temp_output.max())

    ax = axes[0]
    contour1 = ax.contourf(lon_display, latitudes, temp_input,
                           levels=20, cmap='RdYlBu_r',
                           vmin=vmin, vmax=vmax, extend='both')
    ax.contour(lon_display, latitudes, temp_input,
               levels=10, colors='black', linewidths=0.5, alpha=0.3)
    ax.set_title(f'DERNIER INPUT (t)\n{timestamp_input.strftime("%Y-%m-%d %H:%M")}',
                fontsize=12, fontweight='bold', color='blue')
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    ax.grid(True, alpha=0.3)

    stats_in = f'Min: {temp_input.min():.1f}°C | Max: {temp_input.max():.1f}°C | Moy: {temp_input.mean():.1f}°C'
    ax.text(0.5, 0.02, stats_in, transform=ax.transAxes, ha='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), fontsize=9)

    ax = axes[1]
    contour2 = ax.contourf(lon_display, latitudes, temp_output,
                           levels=20, cmap='RdYlBu_r',
                           vmin=vmin, vmax=vmax, extend='both')
    ax.contour(lon_display, latitudes, temp_output,
               levels=10, colors='black', linewidths=0.5, alpha=0.3)
    ax.set_title(f'OUTPUT À PRÉDIRE (t+1)\n{timestamp_output.strftime("%Y-%m-%d %H:%M")}',
                fontsize=12, fontweight='bold', color='red')
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    ax.grid(True, alpha=0.3)

    stats_out = f'Min: {temp_output.min():.1f}°C | Max: {temp_output.max():.1f}°C | Moy: {temp_output.mean():.1f}°C'
    ax.text(0.5, 0.02, stats_out, transform=ax.transAxes, ha='center',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), fontsize=9)

    cbar = fig.colorbar(contour2, ax=axes, orientation='vertical', pad=0.02)
    cbar.set_label('Température (°C)', fontsize=12)

    temp_diff = temp_output - temp_input
    fig.suptitle(f'Comparaison Input vs Output (Séquence {start_idx})\n' +
                f'Écart temporel: 6h | Différence moyenne: {temp_diff.mean():.2f}°C',
                fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
interact(
    plot_temperature,
    t=IntSlider(
        min=0,
        max=len(times)-1,
        step=1,
        value=0,
        description='Timestep:',
        style={'description_width': 'initial'}
    )
)

interact(
    plot_sequence_6to1,
    start_idx=IntSlider(
        min=0,
        max=len(times)-7,
        step=1,
        value=0,
        description='Séquence:',
        style={'description_width': 'initial'}
    )
)

interact(
    plot_input_vs_output,
    start_idx=IntSlider(
        min=0,
        max=len(times)-7,
        step=1,
        value=0,
        description='Séquence:',
        style={'description_width': 'initial'}
    )
)

## Functions we will need for the rest of the project

1- MÉTRIQUES DE PERFORMANCE

In [ ]:
# 1. MÉTRIQUES DE PERFORMANCE

def calculate_metrics(model, dataloader, criterion, device, mean_temp, std_temp,
                     set_name="Test"):
    """
    Calcule toutes les métriques de performance pour un ensemble de données

    Args:
        model: Modèle PyTorch
        dataloader: DataLoader PyTorch
        criterion: Fonction de loss
        device: Device (cpu/cuda)
        mean_temp: Moyenne pour dénormalisation
        std_temp: Écart-type pour dénormalisation
        set_name: Nom de l'ensemble (pour affichage)

    Returns:
        dict: Dictionnaire contenant toutes les métriques
    """

    model.eval()

    all_predictions = []
    all_targets = []
    total_loss = 0

    with torch.no_grad():
        for batch_X, batch_Y in dataloader:
            batch_X = batch_X.to(device)
            batch_Y = batch_Y.to(device)

            # Prédictions
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            total_loss += loss.item()

            # Dénormaliser
            pred_denorm = outputs.cpu().numpy() * std_temp + mean_temp
            target_denorm = batch_Y.cpu().numpy() * std_temp + mean_temp

            all_predictions.append(pred_denorm)
            all_targets.append(target_denorm)

    # Concaténer
    predictions = np.concatenate(all_predictions, axis=0)
    targets = np.concatenate(all_targets, axis=0)

    # Calculer les métriques
    mse = np.mean((predictions - targets) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - targets))

    # R² score
    ss_res = np.sum((targets - predictions) ** 2)
    ss_tot = np.sum((targets - np.mean(targets)) ** 2)
    r2 = 1 - (ss_res / ss_tot)

    # Bias (biais moyen)
    bias = np.mean(predictions - targets)

    # Erreur relative moyenne
    mape = np.mean(np.abs((targets - predictions) / targets)) * 100

    return {
        'loss': total_loss / len(dataloader),
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'bias': bias,
        'mape': mape,
        'predictions': predictions,
        'targets': targets
    }


def evaluate_all_sets(model, train_loader, val_loader, test_loader,
                     criterion, device, mean_temp, std_temp):
    """
    Évalue le modèle sur les 3 ensembles (train, val, test)

    Returns:
        tuple: (train_metrics, val_metrics, test_metrics)
    """
    print(f"\n{'='*60}")
    print("ÉVALUATION COMPLÈTE DU MODÈLE")
    print(f"{'='*60}")

    train_metrics = calculate_metrics(model, train_loader, criterion, device,
                                     mean_temp, std_temp, "Train")
    val_metrics = calculate_metrics(model, val_loader, criterion, device,
                                   mean_temp, std_temp, "Validation")
    test_metrics = calculate_metrics(model, test_loader, criterion, device,
                                    mean_temp, std_temp, "Test")

    return train_metrics, val_metrics, test_metrics


def print_metrics_table(train_metrics, val_metrics, test_metrics):
    """
    Affiche un tableau récapitulatif des métriques
    """
    print(f"\n{'='*70}")
    print(f"{'Métrique':<25} {'Train':<15} {'Validation':<15} {'Test':<15}")
    print(f"{'='*70}")

    metrics_to_show = [
        ('MSE (normalisé)', 'loss', '.6f'),
        ('RMSE (Kelvin)', 'rmse', '.4f'),
        ('RMSE (°C)', 'rmse', '.4f'),
        ('MAE (Kelvin)', 'mae', '.4f'),
        ('R² Score', 'r2', '.4f'),
        ('Bias (Kelvin)', 'bias', '.4f'),
        ('MAPE (%)', 'mape', '.2f')
    ]

    for metric_name, metric_key, fmt in metrics_to_show:
        train_val = train_metrics[metric_key]
        val_val = val_metrics[metric_key]
        test_val = test_metrics[metric_key]
        print(f"{metric_name:<25} {train_val:<15{fmt}} {val_val:<15{fmt}} {test_val:<15{fmt}}")

    print(f"{'='*70}")



2- Visualization functions

In [ ]:
# 2. VISUALISATIONS

def plot_learning_curves(train_losses, val_losses, test_loss, std_temp,
                        save_path='learning_curves.png'):
    """
    Trace les courbes d'apprentissage (Loss et RMSE)

    Args:
        train_losses: Liste des losses d'entraînement par epoch
        val_losses: Liste des losses de validation par epoch
        test_loss: Loss finale sur le test set
        std_temp: Écart-type pour conversion en RMSE
        save_path: Chemin de sauvegarde
    """
    print(f"\n Génération des courbes d'apprentissage...")

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    epochs = range(1, len(train_losses) + 1)

    # Loss normalisée
    ax = axes[0]
    ax.plot(epochs, train_losses, 'b-', label='Train Loss', linewidth=2)
    ax.plot(epochs, val_losses, 'r-', label='Val Loss', linewidth=2)
    ax.axhline(y=test_loss, color='g', linestyle='--',
               label=f'Test Loss: {test_loss:.4f}', linewidth=2)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('MSE Loss (normalisé)', fontsize=12)
    ax.set_title('Courbes d\'apprentissage - Loss', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # RMSE en Kelvin
    ax = axes[1]
    train_rmse = [np.sqrt(loss) * std_temp for loss in train_losses]
    val_rmse = [np.sqrt(loss) * std_temp for loss in val_losses]
    test_rmse = np.sqrt(test_loss) * std_temp

    ax.plot(epochs, train_rmse, 'b-', label='Train RMSE', linewidth=2)
    ax.plot(epochs, val_rmse, 'r-', label='Val RMSE', linewidth=2)
    ax.axhline(y=test_rmse, color='g', linestyle='--',
               label=f'Test RMSE: {test_rmse:.2f}K', linewidth=2)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('RMSE (Kelvin)', fontsize=12)
    ax.set_title('Courbes d\'apprentissage - RMSE', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def plot_predictions_analysis(predictions, targets, mae, r2,
                             save_path='predictions_analysis.png'):
    """
    Analyse des prédictions: scatter plot, distribution erreurs, erreur vs température

    Args:
        predictions: Prédictions du modèle (en Kelvin)
        targets: Valeurs réelles (en Kelvin)
        mae: MAE calculée
        r2: R² score
        save_path: Chemin de sauvegarde
    """
    print(f"\n📊 Génération de l'analyse des prédictions...")

    # Convertir en Celsius
    pred_celsius = (predictions - 273.15).flatten()
    target_celsius = (targets - 273.15).flatten()
    errors = pred_celsius - target_celsius
    abs_errors = np.abs(errors)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Scatter plot
    ax = axes[0]
    ax.scatter(target_celsius, pred_celsius, alpha=0.3, s=1, c='blue')
    ax.plot([target_celsius.min(), target_celsius.max()],
            [target_celsius.min(), target_celsius.max()],
            'r--', linewidth=2, label='Prédiction parfaite')
    ax.set_xlabel('Température réelle (°C)', fontsize=12)
    ax.set_ylabel('Température prédite (°C)', fontsize=12)
    ax.set_title('Prédictions vs Réalité', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.text(0.05, 0.95, f'R² = {r2:.4f}\nMAE = {mae:.2f}K',
            transform=ax.transAxes, fontsize=11, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    # Distribution des erreurs
    ax = axes[1]
    ax.hist(errors, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Erreur nulle')
    ax.axvline(x=errors.mean(), color='g', linestyle='--', linewidth=2,
               label=f'Biais: {errors.mean():.2f}°C')
    ax.set_xlabel('Erreur de prédiction (°C)', fontsize=12)
    ax.set_ylabel('Fréquence', fontsize=12)
    ax.set_title('Distribution des erreurs', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Erreur absolue vs température
    ax = axes[2]
    ax.scatter(target_celsius, abs_errors, alpha=0.3, s=1, c='orange')
    ax.axhline(y=mae, color='r', linestyle='--', linewidth=2,
               label=f'MAE: {mae:.2f}K')
    ax.set_xlabel('Température réelle (°C)', fontsize=12)
    ax.set_ylabel('Erreur absolue (°C)', fontsize=12)
    ax.set_title('Erreur absolue vs Température', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def plot_prediction_map(prediction, target, lats, lons, idx=0,
                       save_path='prediction_map.png'):
    """
    Visualise une prédiction individuelle sur une carte

    Args:
        prediction: Prédiction du modèle (batch)
        target: Valeur réelle (batch)
        lats: Latitudes
        lons: Longitudes
        idx: Index de l'exemple à visualiser
        save_path: Chemin de sauvegarde
    """
    # Extraire et convertir en Celsius
    pred = prediction[idx, 0] - 273.15
    targ = target[idx, 0] - 273.15
    error = pred - targ

    # Convertir longitudes pour affichage
    lon_display = np.where(lons > 180, lons - 360, lons)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Échelle commune pour target et prédiction
    vmin = min(pred.min(), targ.min())
    vmax = max(pred.max(), targ.max())

    # Température réelle
    ax = axes[0]
    im = ax.contourf(lon_display, lats, targ, levels=20, cmap='RdYlBu_r',
                     vmin=vmin, vmax=vmax)
    ax.contour(lon_display, lats, targ, levels=10, colors='black',
               linewidths=0.5, alpha=0.3)
    ax.set_title('Température Réelle', fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude (°E)', fontsize=11)
    ax.set_ylabel('Latitude (°N)', fontsize=11)
    ax.grid(True, alpha=0.3)
    cbar = plt.colorbar(im, ax=ax, label='Température (°C)')

    # Température prédite
    ax = axes[1]
    im = ax.contourf(lon_display, lats, pred, levels=20, cmap='RdYlBu_r',
                     vmin=vmin, vmax=vmax)
    ax.contour(lon_display, lats, pred, levels=10, colors='black',
               linewidths=0.5, alpha=0.3)
    ax.set_title('Température Prédite', fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude (°E)', fontsize=11)
    ax.set_ylabel('Latitude (°N)', fontsize=11)
    ax.grid(True, alpha=0.3)
    cbar = plt.colorbar(im, ax=ax, label='Température (°C)')

    # Erreur
    ax = axes[2]
    # Utiliser une colormap centrée sur 0
    divnorm = TwoSlopeNorm(vmin=error.min(), vcenter=0, vmax=error.max())
    im = ax.contourf(lon_display, lats, error, levels=20, cmap='RdBu_r',
                     norm=divnorm)
    ax.contour(lon_display, lats, error, levels=10, colors='black',
               linewidths=0.5, alpha=0.3)
    mae_local = np.abs(error).mean()
    ax.set_title(f'Erreur de Prédiction\nMAE locale: {mae_local:.2f}°C',
                 fontsize=13, fontweight='bold')
    ax.set_xlabel('Longitude (°E)', fontsize=11)
    ax.set_ylabel('Latitude (°N)', fontsize=11)
    ax.grid(True, alpha=0.3)
    cbar = plt.colorbar(im, ax=ax, label='Erreur (°C)')

    # Statistiques
    stats_text = f'Réel: [{targ.min():.1f}, {targ.max():.1f}]°C\n'
    stats_text += f'Prédit: [{pred.min():.1f}, {pred.max():.1f}]°C\n'
    stats_text += f'Erreur: [{error.min():.1f}, {error.max():.1f}]°C'

    fig.text(0.5, 0.02, stats_text, ha='center', fontsize=10,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.12)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def plot_multiple_examples(predictions, targets, lats, lons, n_examples=3,
                          output_dir='./results'):
    """
    Génère plusieurs exemples de prédictions

    Args:
        predictions: Toutes les prédictions
        targets: Toutes les targets
        lats: Latitudes
        lons: Longitudes
        n_examples: Nombre d'exemples à générer
        output_dir: Dossier de sortie
    """
    print(f"\n Génération de {n_examples} exemples de prédictions...")

    import os
    os.makedirs(output_dir, exist_ok=True)

    # Sélectionner des indices aléatoires
    indices = np.random.choice(len(predictions), n_examples, replace=False)

    for i, idx in enumerate(indices):
        save_path = f'{output_dir}/prediction_example_{i+1}.png'
        plot_prediction_map(predictions, targets, lats, lons, idx, save_path)

3- Save the results

In [ ]:
def save_metrics_to_json(train_metrics, val_metrics, test_metrics,
                        model_name="CNN3DNominal",
                        save_path='./results/metrics.json'):
    """
    Sauvegarde les métriques dans un fichier JSON
    """
    import json
    import os

    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    results = {
        'model_name': model_name,
        'train': {k: float(v) if isinstance(v, (np.floating, float)) else v
                  for k, v in train_metrics.items() if k not in ['predictions', 'targets']},
        'validation': {k: float(v) if isinstance(v, (np.floating, float)) else v
                      for k, v in val_metrics.items() if k not in ['predictions', 'targets']},
        'test': {k: float(v) if isinstance(v, (np.floating, float)) else v
                for k, v in test_metrics.items() if k not in ['predictions', 'targets']}
    }

    with open(save_path, 'w') as f:
        json.dump(results, f, indent=2)

4- Evaluation function

In [ ]:
def evaluate_model_complete(model, train_loader, val_loader, test_loader,
                           criterion, device, mean_temp, std_temp,
                           train_losses, val_losses,
                           lats, lons, model_name="CNN3DNominal",
                           output_dir='./results'):
    """
    Évaluation complète du modèle avec toutes les visualisations

    Args:
        model: Modèle PyTorch entraîné
        train_loader, val_loader, test_loader: DataLoaders
        criterion: Fonction de loss
        device: Device (cpu/cuda)
        mean_temp, std_temp: Paramètres de normalisation
        train_losses, val_losses: Historique des losses
        lats, lons: Coordonnées géographiques
        model_name: Nom du modèle
        output_dir: Dossier de sortie

    Returns:
        tuple: (train_metrics, val_metrics, test_metrics)
    """
    import os
    os.makedirs(output_dir, exist_ok=True)

    # Calculer les métriques
    train_metrics, val_metrics, test_metrics = evaluate_all_sets(
        model, train_loader, val_loader, test_loader,
        criterion, device, mean_temp, std_temp
    )

    #Courbes d'apprentissage
    plot_learning_curves(
        train_losses, val_losses, test_metrics['loss'], std_temp,
        save_path=f'{output_dir}/learning_curves.png'
    )

    # Analyse des prédictions
    plot_predictions_analysis(
        test_metrics['predictions'], test_metrics['targets'],
        test_metrics['mae'], test_metrics['r2'],
        save_path=f'{output_dir}/predictions_analysis.png'
    )

    # Exemples de cartes
    plot_multiple_examples(
        test_metrics['predictions'], test_metrics['targets'],
        lats, lons, n_examples=3, output_dir=output_dir
    )

    # Sauvegarder les métriques
    save_metrics_to_json(
        train_metrics, val_metrics, test_metrics,
        model_name=model_name,
        save_path=f'{output_dir}/metrics.json'
    )

    return train_metrics, val_metrics, test_metrics


5- Training function

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for batch_X, batch_Y in dataloader:
        batch_X = batch_X.to(device, non_blocking=True)
        batch_Y = batch_Y.to(device, non_blocking=True)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping pour stabilité
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch_X, batch_Y in dataloader:
            batch_X = batch_X.to(device, non_blocking=True)
            batch_Y = batch_Y.to(device, non_blocking=True)

            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)

            total_loss += loss.item()

    return total_loss / len(dataloader)

## Nominal experience

The nominal experiment uses a simple 3D CNN architecture with a single convolutional layer. This model serves as a baseline to evaluate the performance improvements of more complex architectures.



---


Nominal model configuration:
•        Architecture : 3D CNN with 1 convolutional layer (1→32 filters, 3×3×3 kernel)

•        Input variable: 2-meter temperature only

•        Temporal input: 6 timesteps (0h, 6h, 12h, 18h, 24h, 30h)

•        Output: Prediction at t+36h (1 timestep)

•        Spatial resolution: 39×57 pixels (France grid)



---



Nominal hyperparameters:
•        Batch size : 16

•        Learning rate : 0.0001

•        Optimizer : Adam (betas=(0.9, 0.999))

•        Scheduler : ReduceLROnPlateau (patience=5, factor=0.5)

•        Loss function : MSE (Mean Squared Error)

•        Epochs: 50

•        Dropout : 0.3


In [ ]:
class CNN3DNominal(nn.Module):
    def __init__(self, dropout=0.3):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels=1,      
            out_channels=32,    
            kernel_size=3,      
            padding=1           
        )

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout3d(p=dropout)

        self.temporal_pool = nn.AdaptiveAvgPool3d((1, 39, 57))

        self.final = nn.Conv3d(32, 1, kernel_size=1)

    def forward(self, x):

        x = self.conv1(x)           
        x = self.relu(x)
        x = self.dropout(x)

        x = self.temporal_pool(x)   
        x = self.final(x)           

        x = x.squeeze(2)            

        return x


#Simple 1-Layer Architecture
```
Input: (batch, 1, 6, 39, 57)
  ↓
Conv3D
• 1 → 32 filters
• Kernel: 3×3×3
• Padding: 1

↓ (batch, 32, 6, 39, 57)
ReLU
↓
Dropout3d (p = 0.3)
↓
AdaptiveAvgPool3D
Output: (1, 39, 57)
6 timesteps → 1 timestep
↓ (batch, 32, 1, 39, 57)
Conv3D (1×1×1)
• 32 → 1 filter
  ↓
Output: (batch, 1, 39, 57)

In [ ]:
# Load data
X_train = np.load('./data/processed/X_train.npy')
Y_train = np.load('./data/processed/Y_train.npy')
X_val = np.load('./data/processed/X_val.npy')
Y_val = np.load('./data/processed/Y_val.npy')
X_test = np.load('./data/processed/X_test.npy')
Y_test = np.load('./data/processed/Y_test.npy')

# Convertir en tensors PyTorch
X_train_tensor = torch.FloatTensor(X_train)
Y_train_tensor = torch.FloatTensor(Y_train)
X_val_tensor = torch.FloatTensor(X_val)
Y_val_tensor = torch.FloatTensor(Y_val)
X_test_tensor = torch.FloatTensor(X_test)
Y_test_tensor = torch.FloatTensor(Y_test)

# Créer les datasets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

# hyperparamètre nominal
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         pin_memory=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        pin_memory=True, num_workers=2)

In [ ]:
model_CNN_nomimal = CNN3DNominal()

# Compter les paramètres
total_params = sum(p.numel() for p in model_CNN_nomimal.parameters())
trainable_params = sum(p.numel() for p in model_CNN_nomimal.parameters() if p.requires_grad)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n Device: {device}")


model = CNN3DNominal(dropout=0.3).to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999)
)
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

In [ ]:
num_epochs = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    current_lr = optimizer.param_groups[0]['lr']
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {current_lr:.6f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_nominal.pth')
        print(f"   Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_nominal.pth'))
print(f"\n Meilleur modèle chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n TEST LOSS FINAL: {test_loss:.6f}")

Evaluation of the nominal model

In [ ]:
# Charger les paramètres de normalisation
norm_params = np.load('./data/processed/norm_params.npy')
mean_temp = norm_params[0]
std_temp = norm_params[1]


# Extraction France
lat_min, lat_max = 42.0, 51.5
lon_min_converted = 355.0
lon_max_converted = 9.0

lat_mask = (lat_values >= lat_min) & (lat_values <= lat_max)
lat_indices = np.where(lat_mask)[0]

lon_mask1 = lon_values >= lon_min_converted
lon_mask2 = lon_values <= lon_max_converted
lon_indices1 = np.where(lon_mask1)[0]
lon_indices2 = np.where(lon_mask2)[0]

# Récupérer les coordonnées
latitudes = lat_values[lat_indices]
lon_part1 = lon_values[lon_indices1]
lon_part2 = lon_values[lon_indices2]
longitudes = np.concatenate([lon_part1, lon_part2])

model_CNN_nomimal.load_state_dict(torch.load('best_model_nominal.pth'))

model_CNN_nomimal.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model_CNN_nomimal,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="CNN3DNominal",
    output_dir='./results/nominal'
)

## Experiences

We realized experiences with different variations :  
  • Input variable  
  • Model architecture  
  • Hyper parameters  

1- Variation of the input variable :  
• First experience : the input variable is the temperature at 2m  
• Second experience : we add the wind speed at 10m to the temperature at 2m for the input

In [20]:
# MODÈLE UNet 3D POUR WEATHER FORECASTING
class WeatherUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.enc1 = self._conv_block(1, 32)
        self.pool1 = nn.MaxPool3d((1, 2, 2))  # (temps, lat, lon)

        self.enc2 = self._conv_block(32, 64)
        self.pool2 = nn.MaxPool3d((1, 2, 2))

        self.enc3 = self._conv_block(64, 128)
        # Decoder
        self.up1 = nn.ConvTranspose3d(128, 64, kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.dec1 = self._conv_block(128, 64)  # concat 64+64

        self.up2 = nn.ConvTranspose3d(64, 32, kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.dec2 = self._conv_block(64, 32)   # concat 32+32

        self.temporal_pool = nn.AdaptiveAvgPool3d((1, 39, 57))  # Fixe le shape de sortie
        self.final = nn.Conv3d(32, 1, kernel_size=1)

    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv3d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm3d(out_ch),
            nn.ReLU(inplace=True)
        )

    def _crop_to_match(self, src, target):
        # Src = tenseur à croper, target = référence de taille
        diff_t = src.shape[2] - target.shape[2]
        diff_h = src.shape[3] - target.shape[3]
        diff_w = src.shape[4] - target.shape[4]
        crop_t = diff_t // 2
        crop_h = diff_h // 2
        crop_w = diff_w // 2
        return src[
            :,
            :,
            crop_t:src.shape[2] - (diff_t - crop_t),
            crop_h:src.shape[3] - (diff_h - crop_h),
            crop_w:src.shape[4] - (diff_w - crop_w)
        ]

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        e3 = self.enc3(p2)
        # Decoder up + crop + concat
        u1 = self.up1(e3)
        e2_crop = self._crop_to_match(e2, u1)
        d1 = self.dec1(torch.cat([u1, e2_crop], dim=1))
        u2 = self.up2(d1)
        e1_crop = self._crop_to_match(e1, u2)
        d2 = self.dec2(torch.cat([u2, e1_crop], dim=1))
        # Sortie finale
        out = self.temporal_pool(d2)
        out = self.final(out)
        out = out.view(out.size(0), 1, 39, 57)  # (batch, 1, 39, 57)
        return out

In [ ]:
model_Unet_temp_2m = WeatherUNet()

# Calculate the number of parameters
total_params = sum(p.numel() for p in model_Unet_temp_2m.parameters())
trainable_params = sum(p.numel() for p in model_Unet_temp_2m.parameters() if p.requires_grad)

print(f"\n Paramètres du modèle:")
print(f"  - Total: {total_params:,}")
print(f"  - Entraînables: {trainable_params:,}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_Unet_temp_2m = model_Unet_temp_2m.to(device)

criterion = nn.MSELoss()

learning_rate = 0.0001
optimizer = torch.optim.Adam(model_Unet_temp_2m.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
X_train = np.load('./data/processed/X_train.npy')
Y_train = np.load('./data/processed/Y_train.npy')
X_val = np.load('./data/processed/X_val.npy')
Y_val = np.load('./data/processed/Y_val.npy')
X_test = np.load('./data/processed/X_test.npy')
Y_test = np.load('./data/processed/Y_test.npy')

X_train_tensor = torch.FloatTensor(X_train)
Y_train_tensor = torch.FloatTensor(Y_train)
X_val_tensor = torch.FloatTensor(X_val)
Y_val_tensor = torch.FloatTensor(Y_val)
X_test_tensor = torch.FloatTensor(X_test)
Y_test_tensor = torch.FloatTensor(Y_test)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         pin_memory=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       pin_memory=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        pin_memory=True, num_workers=2)

In [ ]:
best_val_loss = float('inf')
patience = 10
patience_counter = 0
num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n -- Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model_Unet_temp_2m, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model_Unet_temp_2m, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model_Unet_temp_2m.state_dict(), 'best_model_Unet_temp_2m.pth')
        print(f" Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model_Unet_temp_2m.load_state_dict(torch.load('best_model_Unet_temp_2m.pth'))
print(f"\n Meilleur modèle model_Unet_temp_2m chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model_Unet_temp_2m, test_loader, criterion, device)
print(f"\n Test Loss: {test_loss:.6f}")

In [ ]:
# Charger les paramètres de normalisation
norm_params = np.load('./data/processed/norm_params.npy')
mean_temp = norm_params[0]
std_temp = norm_params[1]


# Extraction France
lat_min, lat_max = 42.0, 51.5
lon_min_converted = 355.0
lon_max_converted = 9.0

lat_mask = (lat_values >= lat_min) & (lat_values <= lat_max)
lat_indices = np.where(lat_mask)[0]

lon_mask1 = lon_values >= lon_min_converted
lon_mask2 = lon_values <= lon_max_converted
lon_indices1 = np.where(lon_mask1)[0]
lon_indices2 = np.where(lon_mask2)[0]

# Récupérer les coordonnées
latitudes = lat_values[lat_indices]
lon_part1 = lon_values[lon_indices1]
lon_part2 = lon_values[lon_indices2]
longitudes = np.concatenate([lon_part1, lon_part2])

model_Unet_temp_2m.load_state_dict(torch.load('best_model_Unet_temp_2m.pth'))

model_Unet_temp_2m.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model_Unet_temp_2m,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherUNet",
    output_dir='./results/WeatherUNet_temp_variable'
)

Second experience :  
• Input variables : temperature at 2m + wind speed at 10m

In [ ]:
#Data preparation for 2 variables 

dataset_path = "gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr"

try:
    # Connexion anonyme
    fs = gcsfs.GCSFileSystem(token='anon')
    mapper = fs.get_mapper(dataset_path)
    ds = xr.open_zarr(mapper, chunks={'time': 24}, consolidated=True)

    # Charger 2 variables : température à 2m + vitesse du vent à 10m
    temperature_data = ds['2m_temperature']
    wind_speed_data = ds['10m_wind_speed']

    # Concaténer les deux variables
    data_wind_tem = xr.concat([temperature_data, wind_speed_data], dim='variable')
    data_wind_tem = data_wind_tem.assign_coords(variable=["t2m", "wind10m"])

    # EXTRACTION FRANCE
    lat_min, lat_max = 42.0, 51.5
    lon_min_converted = 355.0  # -5 + 360
    lon_max_converted = 9.0

    lat_values = data_wind_tem.latitude.values
    lon_values = data_wind_tem.longitude.values

    # Trouver les indices
    lat_mask = (lat_values >= lat_min) & (lat_values <= lat_max)
    lat_indices = np.where(lat_mask)[0]

    lon_mask1 = lon_values >= lon_min_converted  # 355° à 360°
    lon_mask2 = lon_values <= lon_max_converted  # 0° à 9°
    lon_indices1 = np.where(lon_mask1)[0]
    lon_indices2 = np.where(lon_mask2)[0]

    # Extraire en 2 parties
    france_part1 = data_wind_tem.isel(latitude=lat_indices, longitude=lon_indices1)
    france_part2 = data_wind_tem.isel(latitude=lat_indices, longitude=lon_indices2)

    france_data = xr.concat([france_part1, france_part2], dim='longitude')

    # Résultats
    nbytes = france_data.nbytes
    size_str = f"{nbytes / 1e9:.2f} GB" if nbytes > 1e9 else f"{nbytes / 1e6:.2f} MB"
    print(f"\nTaille mémoire: {size_str}")

except Exception as e:
    print(f"\n Erreur: {e}")

In [15]:
france_data

<xarray.DataArray '2m_temperature' (variable: 2, time: 92044, latitude: 39,
                                    longitude: 57)> Size: 2GB
dask.array<concatenate, shape=(2, 92044, 39, 57), dtype=float32, chunksize=(1, 24, 39, 37), chunktype=numpy.ndarray>
Coordinates:
  * variable   (variable) <U7 56B 't2m' 'wind10m'
  * time       (time) datetime64[ns] 736kB 1959-01-01 ... 2021-12-31T18:00:00
  * latitude   (latitude) float32 156B 51.5 51.25 51.0 50.75 ... 42.5 42.25 42.0
  * longitude  (longitude) float32 228B 355.0 355.2 355.5 355.8 ... 8.5 8.75 9.0
Attributes:
    long_name:   2 metre temperature
    short_name:  t2m
    units:       K

In [16]:
# Données de test : année 2020 ET 2021
test_data = france_data.sel(time=slice("2020-01-01", "2021-12-31"))

# Données validation : année 2018
validation_data = france_data.sel(time=slice("2018-01-01", "2018-12-31"))

# Données train : années 2011-2016
train_data = france_data.sel(time=slice("2011-01-01", "2016-12-31"))

In [18]:
INPUT_TIMESTEPS = 6   # 6 timesteps en entrée (30h d'historique avec mesures toutes les 6h)
OUTPUT_TIMESTEPS = 1  # 1 timestep en sortie (+6h de prédiction)

In [ ]:
#Function to create sequences for multivariable input
def create_sequences_multivar(data, input_steps=6, output_offset=1):
    """
    Créer séquences pour données multi-variables

    Args:
        data: xarray DataArray de shape (n_vars, time, lat, lon)
        input_steps: nombre de timesteps d'entrée (6)
        output_offset: décalage pour prédiction (1 = +6h)

    Returns:
        X: (samples, n_vars, input_steps, lat, lon)
        Y: (samples, 1, lat, lon)  # Seulement température en sortie
    """
    # Charger en mémoire
    data_np = data.values  # Shape: (n_vars, time, lat, lon)

    n_vars = data_np.shape[0]
    n_times = data_np.shape[1]
    lat = data_np.shape[2]
    lon = data_np.shape[3]

    n_samples = n_times - input_steps - output_offset + 1

    X = []
    Y = []

    for i in range(n_samples):
        # Input: TOUTES les variables sur input_steps timesteps
        # Shape pour un sample: (n_vars, input_steps, lat, lon)
        X.append(data_np[:, i:i+input_steps, :, :])

        # Output: SEULEMENT température (variable 0) au timestep futur
        # Shape pour un sample: (lat, lon)
        Y.append(data_np[0, i+input_steps+output_offset-1, :, :])

    X = np.array(X)  # (samples, n_vars, input_steps, lat, lon)
    Y = np.array(Y)  # (samples, lat, lon)

    # Ajouter dimension channel à Y
    Y = np.expand_dims(Y, axis=1)  # (samples, 1, lat, lon)

    return X, Y

In [19]:
# Créer les séquences pour chaque split
X_train_raw, Y_train_raw = create_sequences_multivar(
    train_data,
    input_steps=INPUT_TIMESTEPS,
    output_offset=OUTPUT_TIMESTEPS
)

X_val_raw, Y_val_raw = create_sequences_multivar(
    validation_data,
    input_steps=INPUT_TIMESTEPS,
    output_offset=OUTPUT_TIMESTEPS
)

X_test_raw, Y_test_raw = create_sequences_multivar(
    test_data,
    input_steps=INPUT_TIMESTEPS,
    output_offset=OUTPUT_TIMESTEPS
)

In [ ]:
#Normalization

# Variable 0: Température
temp_train = X_train_raw[:, 0, :, :, :] 
mean_temp = temp_train.mean()
std_temp = temp_train.std()

# Variable 1: Vent
wind_train = X_train_raw[:, 1, :, :, :]  # Extraire canal vent
mean_wind = wind_train.mean()
std_wind = wind_train.std()

def normalize_X_multivar(X, mean_temp, std_temp, mean_wind, std_wind):
    """Normaliser les entrées multi-variables"""
    X_norm = X.copy()
    X_norm[:, 0, :, :, :] = (X[:, 0, :, :, :] - mean_temp) / std_temp  # Température
    X_norm[:, 1, :, :, :] = (X[:, 1, :, :, :] - mean_wind) / std_wind  # Vent
    return X_norm

def normalize_Y(Y, mean_temp, std_temp):
    """Normaliser la sortie (température)"""
    return (Y - mean_temp) / std_temp

# Appliquer normalisation
X_train_norm = normalize_X_multivar(X_train_raw, mean_temp, std_temp, mean_wind, std_wind)
Y_train_norm = normalize_Y(Y_train_raw, mean_temp, std_temp)

X_val_norm = normalize_X_multivar(X_val_raw, mean_temp, std_temp, mean_wind, std_wind)
Y_val_norm = normalize_Y(Y_val_raw, mean_temp, std_temp)

X_test_norm = normalize_X_multivar(X_test_raw, mean_temp, std_temp, mean_wind, std_wind)
Y_test_norm = normalize_Y(Y_test_raw, mean_temp, std_temp)

# Sauvegarder les statistiques pour dénormalisation
stats = {
    'mean_temp': float(mean_temp),
    'std_temp': float(std_temp),
    'mean_wind': float(mean_wind),
    'std_wind': float(std_wind)
}

In [22]:
# Récupérer les données pour visualisation
longitudes = france_data.longitude.values
latitudes = france_data.latitude.values
times = train_data.time.values

# Extraire la température pour visualisation
data_np_temp = X_train_raw[:, 0, 0, :, :]  # Premier timestep, canal température

# Convertir les longitudes > 180
lon_display = np.where(longitudes > 180, longitudes - 360, longitudes)

In [ ]:
# Convertir en tensors PyTorch
X_train_tensor = torch.FloatTensor(X_train_norm)
Y_train_tensor = torch.FloatTensor(Y_train_norm)

X_val_tensor = torch.FloatTensor(X_val_norm)
Y_val_tensor = torch.FloatTensor(Y_val_norm)

X_test_tensor = torch.FloatTensor(X_test_norm)
Y_test_tensor = torch.FloatTensor(Y_test_norm)

# Créer datasets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, Y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

# Créer dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

UNet architechture for 2 variables

In [ ]:
# MODÈLE U-NET 3D - VERSION FINALE CORRIGÉE
import torch
import torch.nn as nn
import torch.nn.functional as F

class WeatherUNet_multivar(nn.Module):
    def __init__(self, in_channels=2):
        super(WeatherUNet_multivar, self).__init__()

        # ENCODER
        self.enc1 = nn.Conv3d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.enc1_2 = nn.Conv3d(32, 32, kernel_size=3, padding=1)
        self.bn1_2 = nn.BatchNorm3d(32)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        self.enc2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.enc2_2 = nn.Conv3d(64, 64, kernel_size=3, padding=1)
        self.bn2_2 = nn.BatchNorm3d(64)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        self.enc3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.enc3_2 = nn.Conv3d(128, 128, kernel_size=3, padding=1)
        self.bn3_2 = nn.BatchNorm3d(128)

        # DECODER
        self.up1 = nn.ConvTranspose3d(128, 64, kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.dec1 = nn.Conv3d(128, 64, kernel_size=3, padding=1)
        self.bn_dec1 = nn.BatchNorm3d(64)
        self.dec1_2 = nn.Conv3d(64, 64, kernel_size=3, padding=1)
        self.bn_dec1_2 = nn.BatchNorm3d(64)

        self.up2 = nn.ConvTranspose3d(64, 32, kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.dec2 = nn.Conv3d(64, 32, kernel_size=3, padding=1)
        self.bn_dec2 = nn.BatchNorm3d(32)
        self.dec2_2 = nn.Conv3d(32, 32, kernel_size=3, padding=1)
        self.bn_dec2_2 = nn.BatchNorm3d(32)

        self.temporal_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.out = nn.Conv3d(32, 1, kernel_size=1)

        print(f"U-Net créé avec {in_channels} canaux")

    def forward(self, x):
        # Sauvegarder la taille d'entrée
        input_size = x.shape
        target_h, target_w = input_size[3], input_size[4]

        # Encoder
        x1 = F.relu(self.bn1(self.enc1(x)))
        x1 = F.relu(self.bn1_2(self.enc1_2(x1)))
        p1 = self.pool1(x1)

        x2 = F.relu(self.bn2(self.enc2(p1)))
        x2 = F.relu(self.bn2_2(self.enc2_2(x2)))
        p2 = self.pool2(x2)

        x3 = F.relu(self.bn3(self.enc3(p2)))
        x3 = F.relu(self.bn3_2(self.enc3_2(x3)))

        # Decoder
        u1 = self.up1(x3)

        if u1.shape[2:] != x2.shape[2:]:
            diff_d = x2.size(2) - u1.size(2)
            diff_h = x2.size(3) - u1.size(3)
            diff_w = x2.size(4) - u1.size(4)

            x2 = x2[:, :,
                   diff_d//2 : diff_d//2 + u1.size(2),
                   diff_h//2 : diff_h//2 + u1.size(3),
                   diff_w//2 : diff_w//2 + u1.size(4)]

        c1 = torch.cat([u1, x2], dim=1)
        d1 = F.relu(self.bn_dec1(self.dec1(c1)))
        d1 = F.relu(self.bn_dec1_2(self.dec1_2(d1)))

        u2 = self.up2(d1)

        if u2.shape[2:] != x1.shape[2:]:
            diff_d = x1.size(2) - u2.size(2)
            diff_h = x1.size(3) - u2.size(3)
            diff_w = x1.size(4) - u2.size(4)

            x1 = x1[:, :,
                   diff_d//2 : diff_d//2 + u2.size(2),
                   diff_h//2 : diff_h//2 + u2.size(3),
                   diff_w//2 : diff_w//2 + u2.size(4)]

        c2 = torch.cat([u2, x1], dim=1)
        d2 = F.relu(self.bn_dec2(self.dec2(c2)))
        d2 = F.relu(self.bn_dec2_2(self.dec2_2(d2)))

        d2 = self.temporal_pool(d2)

        out = self.out(d2)
        out = out.squeeze(2) 

        if out.shape[2] != target_h or out.shape[3] != target_w:
            out = F.interpolate(out, size=(target_h, target_w),
                              mode='bilinear', align_corners=False)

        return out

In [ ]:
model = WeatherUNet_multivar(in_channels=2)  # 2 variables
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.L1Loss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
num_epochs = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    current_lr = optimizer.param_groups[0]['lr']
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {current_lr:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_WeatherUNet_multivar.pth')
        print(f"   Nouveau meilleur modèle sauvegardé!")

print("\n Entraînement terminé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_WeatherUNet_multivar.pth'))
print(f"\n Meilleur modèle chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n TEST LOSS FINAL: {test_loss:.6f}")

In [ ]:
model.load_state_dict(torch.load('best_model_WeatherUNet_multivar.pth'))

model.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherUNet_multivar",
    output_dir='./results/WeatherUNet_multivar'
)

Since the model performed better with two variables in input rather than one, we will continue the following experiences using two variables as input

# 2- Variation of the model architecture

2.1- ResNet-LSTM

In [ ]:
class ResidualBlock3D(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock3D, self).__init__()

        self.conv1 = nn.Conv3d(
            in_channels, out_channels,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm3d(out_channels)

        self.conv2 = nn.Conv3d(
            out_channels, out_channels,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm3d(out_channels)

        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x

        # First conv + BN + ReLU
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # Second conv + BN
        out = self.conv2(out)
        out = self.bn2(out)

        # Downsample residual if needed
        if self.downsample is not None:
            residual = self.downsample(x)

        # Skip connection (c'est ça qui fait un ResNet!)
        out += residual
        out = self.relu(out)

        return out


class WeatherResNetLSTM_multivar(nn.Module):
    """
    Architecture hybride ResNet-LSTM pour prévision météo multivariée

    Architecture:
    1. ResNet 3D encoder : extraction features spatio-temporelles avec skip connections
    2. LSTM : modélisation dynamique temporelle
    3. Decoder : reconstruction prédiction spatiale

    Input: (batch, 2, 6, 39, 57)
           - 2 variables : température (2m) + vent (10m)
           - 6 timesteps historiques
           - 39×57 grille spatiale France

    Output: (batch, 1, 39, 57)
            - 1 variable : température prédite
            - 1 timestep : prédiction à t+6h
    """
    def __init__(self, in_channels=2):
        super(WeatherResNetLSTM_multivar, self).__init__()

        self.in_channels = in_channels
        self.conv1 = nn.Conv3d(
            in_channels, 32,
            kernel_size=3,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm3d(32)
        self.relu = nn.ReLU(inplace=True)

        # Layer 1 : Residual blocks sans downsampling
        self.layer1 = self._make_layer(32, 32, blocks=2)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2))  # Downsample spatial seulement

        # Layer 2 : Residual blocks avec augmentation de canaux
        downsample2 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm3d(64)
        )
        self.layer2 = self._make_layer(32, 64, blocks=2, stride=1, downsample=downsample2)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 2, 2))

        # Layer 3 : Residual blocks finaux
        downsample3 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm3d(128)
        )
        self.layer3 = self._make_layer(64, 128, blocks=2, stride=1, downsample=downsample3)


        # Après 2 poolings 2x2 : 39→19→9, 57→28→14
        self.lstm_input_size = 128 * 9 * 14  # channels * height * width

        self.lstm = nn.LSTM(
            input_size=self.lstm_input_size,
            hidden_size=512,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )


        self.decoder = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 39 * 57) 
        )

    def _make_layer(self, in_channels, out_channels, blocks, stride=1, downsample=None):
        """
        Créer une couche résiduelle avec plusieurs blocs
        """
        layers = []

        # Premier bloc (peut avoir downsample)
        layers.append(ResidualBlock3D(in_channels, out_channels, stride, downsample))

        # Blocs suivants (pas de downsample)
        for _ in range(1, blocks):
            layers.append(ResidualBlock3D(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        """
        Args:
            x: (batch, channels, time, height, width) = (N, 2, 6, 39, 57)

        Returns:
            out: (batch, 1, height, width) = (N, 1, 39, 57)
        """
        batch_size = x.size(0)
        target_h, target_w = x.size(3), x.size(4)


        # Stem
        x = self.conv1(x)           
        x = self.bn1(x)
        x = self.relu(x)

        # Layer 1 + Pool
        x = self.layer1(x)          
        x = self.pool1(x)          

        # Layer 2 + Pool
        x = self.layer2(x)          
        x = self.pool2(x)           

        # Layer 3
        x = self.layer3(x)          


        x = x.permute(0, 2, 1, 3, 4)  
        x = x.contiguous().view(batch_size, 6, -1) 


        lstm_out, (h_n, c_n) = self.lstm(x)  

        last_output = lstm_out[:, -1, :]  


        output = self.decoder(last_output) 

        output = output.view(batch_size, 1, 39, 57)  


        if output.shape[2] != target_h or output.shape[3] != target_w:
            output = F.interpolate(
                output,
                size=(target_h, target_w),
                mode='bilinear',
                align_corners=False
            )

        return output 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_WeatherResNetLSTM_multivar = WeatherResNetLSTM_multivar(in_channels=2).to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(model_WeatherResNetLSTM_multivar.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
best_val_loss = float('inf')
patience = 10
patience_counter = 0

num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n -- Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model_WeatherResNetLSTM_multivar, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model_WeatherResNetLSTM_multivar, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model_WeatherResNetLSTM_multivar.state_dict(), 'best_model_WeatherResNetLSTM_multivar.pth')
        print(f" Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model_WeatherResNetLSTM_multivar.load_state_dict(torch.load('best_model_WeatherResNetLSTM_multivar.pth'))
print(f"\nMeilleur modèle model_WeatherResNetLSTM_multivar chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model_WeatherResNetLSTM_multivar, test_loader, criterion, device)

In [ ]:
model_WeatherResNetLSTM_multivar.load_state_dict(torch.load('best_model_WeatherResNetLSTM_multivar.pth'))

model_WeatherResNetLSTM_multivar.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model_WeatherResNetLSTM_multivar,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherResNetLSTM_multivar",
    output_dir='./results/WeatherResNetLSTM_multivar'
)

In [ ]:
2.2- Conv-LSTM

In [ ]:
class ConvLSTMCell(nn.Module):
    """
    Cellule ConvLSTM : combine convolution spatiale + LSTM temporel
    """
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias
        self.conv = nn.Conv2d(
            in_channels=self.input_dim + self.hidden_dim,
            out_channels=4 * self.hidden_dim,
            kernel_size=self.kernel_size,
            padding=self.padding,
            bias=self.bias
        )

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        # Concatenate input and hidden state
        combined = torch.cat([input_tensor, h_cur], dim=1)

        # Convolution for all gates
        combined_conv = self.conv(combined)

        # Split into 4 gates
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)

        # Apply activations
        i = torch.sigmoid(cc_i)  # Input gate
        f = torch.sigmoid(cc_f)  # Forget gate
        o = torch.sigmoid(cc_o)  # Output gate
        g = torch.tanh(cc_g)     # Cell gate

        # Update cell state
        c_next = f * c_cur + i * g

        # Update hidden state
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        device = self.conv.weight.device
        return (
            torch.zeros(batch_size, self.hidden_dim, height, width, device=device),
            torch.zeros(batch_size, self.hidden_dim, height, width, device=device)
        )

class ConvLSTM(nn.Module):
    """
    Module ConvLSTM avec plusieurs couches empilées
    """
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=True, bias=True):
        super(ConvLSTM, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim
            cell_list.append(
                ConvLSTMCell(
                    input_dim=cur_input_dim,
                    hidden_dim=self.hidden_dim,
                    kernel_size=self.kernel_size,
                    bias=self.bias
                )
            )

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Args:
            input_tensor: (batch, time, channels, height, width)
            hidden_state: None or tuple of (h, c) for each layer

        Returns:
            layer_output: output of last layer (batch, time, hidden_dim, H, W)
            last_state: (h, c) of last layer at last timestep
        """
        if not self.batch_first:
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        batch_size, seq_len, _, height, width = input_tensor.size()

        # Initialize hidden state
        if hidden_state is None:
            hidden_state = self._init_hidden(batch_size, (height, width))

        layer_output_list = []
        last_state_list = []

        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx]
            output_inner = []

            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](
                    input_tensor=cur_layer_input[:, t, :, :, :],
                    cur_state=[h, c]
                )
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        return layer_output_list[-1], last_state_list[-1]

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

class WeatherConvLSTM_multivar(nn.Module):
    """
    Architecture ConvLSTM pour prévision météo multivariée

    Input: (batch, 2, 6, 39, 57)
           - 2 variables : température (2m) + vent (10m)
           - 6 timesteps historiques
           - 39×57 grille spatiale France

    Output: (batch, 1, 39, 57)
            - 1 variable : température prédite
            - 1 timestep : prédiction à t+6h
    """
    def __init__(self, in_channels=2, hidden_dims=[64, 128, 64],
                 kernel_size=3, num_layers=2):
        super(WeatherConvLSTM_multivar, self).__init__()

        self.in_channels = in_channels
        self.hidden_dims = hidden_dims
        self.spatial_encoder = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dims[0], kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_dims[0]),
            nn.ReLU(),
            nn.Conv2d(hidden_dims[0], hidden_dims[0], kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_dims[0]),
            nn.ReLU()
        )
        self.convlstm = ConvLSTM(
            input_dim=hidden_dims[0],
            hidden_dim=hidden_dims[1],
            kernel_size= kernel_size,
            num_layers=num_layers,
            batch_first=True,
            bias=True
        )
        self.spatial_decoder = nn.Sequential(
            nn.Conv2d(hidden_dims[1], hidden_dims[2], kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_dims[2]),
            nn.ReLU(),
            nn.Conv2d(hidden_dims[2], hidden_dims[2], kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_dims[2]),
            nn.ReLU(),
            nn.Conv2d(hidden_dims[2], 1, kernel_size=1)
        )

    def forward(self, x):
        """
        Args:
            x: (batch, channels, time, height, width) = (N, 2, 6, 39, 57)

        Returns:
            out: (batch, 1, height, width) = (N, 1, 39, 57)
        """
        batch_size, channels, seq_len, height, width = x.size()

        target_h, target_w = height, width
        encoded_frames = []
        for t in range(seq_len):
            frame = x[:, :, t, :, :]
            encoded = self.spatial_encoder(frame)
            encoded_frames.append(encoded)
        encoded_sequence = torch.stack(encoded_frames, dim=1)
        lstm_output, last_state = self.convlstm(encoded_sequence)
        last_frame = lstm_output[:, -1, :, :, :]

        output = self.spatial_decoder(last_frame)
        if output.shape[2] != target_h or output.shape[3] != target_w:
            output = F.interpolate(
                output,
                size=(target_h, target_w),
                mode='bilinear',
                align_corners=False
            )

        return output  # (batch, 1, 39, 57)

In [ ]:
model = WeatherConvLSTM_multivar(in_channels=2) 
model = model.to(device)
criterion = nn.MSELoss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
num_epochs = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    current_lr = optimizer.param_groups[0]['lr']
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {current_lr:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_WeatherUNet_multivar.pth')
        print(f"Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_WeatherUNet_multivar.pth'))
print(f"\n Meilleur modèle chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n TEST LOSS FINAL: {test_loss:.6f}")

In [ ]:
model.load_state_dict(torch.load('best_model_WeatherUNet_multivar.pth'))

model.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherUNet_multivar",
    output_dir='./results/WeatherUNet_multivar'
)

# 3-Variation of hyper-parameters

Variation of the loss function and the optimizer

3.1- MAE + Adam

In [ ]:
model = WeatherUNet_multivar(in_channels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.L1Loss()
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
num_epochs = 50
best_val_loss = float('inf')
patience = 10
patience_counter = 0

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)

    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)

    # Affichage
    current_lr = optimizer.param_groups[0]['lr']
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {current_lr:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_WeatherUNet_multivar.pth')
        print(f"   Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_WeatherUNet_multivar.pth'))
print(f"\n Meilleur modèle chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n TEST LOSS FINAL: {test_loss:.6f}")

3.2- SGD + MSE

In [ ]:
# Configuration device et optimisation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
best_val_loss = float('inf')
patience = 10
patience_counter = 0

num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n -- Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)
    # Affichage
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_WeatherUNET_multivar_SGD_MSE.pth')
        print(f"  Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_WeatherUNET_multivar_SGD_MSE.pth'))
print(f"\nMeilleur modèle best_model_WeatherUNET_multivar_SGD_MSE chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n Test Loss: {test_loss:.6f}")

In [ ]:
model.load_state_dict(torch.load('best_model_WeatherUNET_multivar_SGD_MSE.pth'))

model.to(device)

train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherUNET_multivar_SGD_MSE",
    output_dir='./results/WeatherUNET_multivar_SGD_MSE'
)

SGD + MAE

In [ ]:
model = WeatherUNet_multivar(in_channels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.L1Loss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
best_val_loss = float('inf')
patience = 10
patience_counter = 0
num_epochs = 50
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\n -- Epoch {epoch+1}/{num_epochs}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    # Entraînement
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    # Validation
    val_loss = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)

    # Scheduler
    scheduler.step(val_loss)
    # Affichage
    print(f"  Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Sauvegarder le meilleur modèle
        torch.save(model.state_dict(), 'best_model_WeatherUNET_multivar_SGD_MAE.pth')
        print(f"Nouveau meilleur modèle sauvegardé!")

# Charger le meilleur modèle
model.load_state_dict(torch.load('best_model_WeatherUNET_multivar_SGD_MAE.pth'))
print(f"\nMeilleur modèle best_model_WeatherUNET_multivar_SGD_MAE chargé (Val Loss: {best_val_loss:.6f})")

# Évaluation finale sur test set
test_loss = validate(model, test_loader, criterion, device)
print(f"\n Test Loss: {test_loss:.6f}")

In [ ]:
model.load_state_dict(torch.load('best_model_WeatherUNET_multivar_SGD_MAE.pth'))
model.to(device)
train_metrics, val_metrics, test_metrics = evaluate_model_complete(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
    mean_temp=mean_temp,
    std_temp=std_temp,
    train_losses=train_losses,
    val_losses=val_losses,
    lats=latitudes,
    lons=longitudes,
    model_name="WeatherUNET_multivar_SGD_MAE",
    output_dir='./results/WeatherUNET_multivar_SGD_MAE'
)

## Conclusion

The set of experiments shows that adding a dynamic variable such as 10-meter wind speed consistently improves temperature forecasting accuracy, slightly reducing the RMSE and stabilizing the training process.

Regarding the architectures, the 3D U-Net appears to be the most robust model: it converges quickly, generalizes well, and achieves the best overall performance ahead of the Conv-LSTM and the ResNet-LSTM.

Finally, among the tested hyperparameters, the Adam + MSE combination is the most effective; MAE losses and the SGD optimizer lead to a clear degradation in performance.

Overall, the results confirm that integrating multivariate information and using deep convolutional architectures are appropriate and effective approaches for short-term weather forecasting on ERA5 France.